# La Compagnie du Vent Challenge 

In [1]:
# pacakge import
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
from datetime import timedelta
import gc
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

In [2]:
def create_df_parc_data(folder_adr,list_num_parc,list_date_parc):
    """
    a function to load (and concatene) informations from Parc_XX.csv
    """
    df_parc_data = pd.DataFrame()
    for num_parc in list_num_parc:
        for date_parc in list_date_parc:
            df_parc_data = df_parc_data.append(pd.read_csv(folder_adr+"//Parc%s_%s.csv"%(num_parc,date_parc),sep=";",decimal=','),ignore_index=True)
    df_parc_data["Date"] = pd.to_datetime(df_parc_data["Date"],format = "%d/%m/%Y %H:%M")
    return df_parc_data

def load_parc_data(folder_adr,list_num_parc,list_date_parc,list_col_parc_data_to_keep):
    """
    keep only fonctionnement == 1 and selection columns
    """
    df_parc_data = create_df_parc_data(folder_adr,list_num_parc,list_date_parc)
    # keep only fonctionnment = 1
    df_parc_data =  df_parc_data[df_parc_data["Fonctionnement"]==1]
    print(df_parc_data.shape)
    return df_parc_data[list_col_parc_data_to_keep]

def add_feature_timestamp(df_parc_data):
    """
    create time features
    """
    df_parc_data["Month"] = df_parc_data["Date"].dt.month
    df_parc_data["Day"] = df_parc_data["Date"].dt.day
    df_parc_data["Hour"] = df_parc_data["Date"].dt.hour
    df_parc_data["Weekday"] = df_parc_data["Date"].dt.weekday
    df_parc_data["Date_hour_int"] = df_parc_data["Date"].dt.year*10**6  + df_parc_data["Date"].dt.month*10**4 +\
    df_parc_data["Date"].dt.day*100 + df_parc_data["Date"].dt.hour
    df_parc_data["nb_hour"] = df_parc_data["Date"].apply(lambda x: np.floor((x- datetime.datetime(2015,1,1)).total_seconds()/3600))
    return 
 
def min_to_hour(df_parc_data):
    """
    convert minute dataframe to hourly dataframe
    """
    df_parc_data_hour = df_parc_data.groupby(["Date_hour_int","Eolienne"]).mean()
    return df_parc_data_hour.reset_index()

def get_eolienne_list(df_parc_data):
    """
    get all eolienne name in the df
    """
    return df_parc_data["Eolienne"].drop_duplicates().tolist()

def create_df_meteo_from_list_grille(folder_adr,list_grille):
    """
    a function to load (and concatenate) informations from PrevMeteo_GrilleXX.xlsx
    """
    df_meteo  = pd.DataFrame()
    for grille_C in list_grille:
        df_meteo_tmp =  pd.read_excel(folder_adr +'/PrevMeteo_Grille%s.xlsx'%(grille_C),sep=';')
        df_meteo_tmp["grille"] = grille_C
        df_meteo = df_meteo.append(df_meteo_tmp,ignore_index=True)
    df_meteo["date"] = pd.to_datetime(df_meteo["date"],format = "%Y-%m-%d %H:%M:%S")
    df_meteo.rename(columns= {"date":"Date"},inplace=True)
    return df_meteo

def meteo_grill_merge(df_meteo,feature_list,join_key):
    """
    merge meteo data by grille
    """
    grille_list = df_meteo["grille"].drop_duplicates().tolist() 
    all_feature_list = feature_list + join_key
    grille_name = grille_list[0]
    index = df_meteo["grille"] == grille_name
    df_meteo_merged = df_meteo.loc[index,all_feature_list]
    df_meteo_merged.columns =  [x +"_"+ str(grille_name) for x in feature_list] + join_key
    if len(grille_list)==1:
        return df_meteo_merged
    for grille_name in grille_list[1:]:
        index = df_meteo["grille"] == grille_name
        df_meteo_merged_p = df_meteo.loc[index,all_feature_list]
        df_meteo_merged_p.columns =  [x +"_"+ str(grille_name) for x in feature_list] + join_key
        df_meteo_merged =  pd.merge(df_meteo_merged,df_meteo_merged_p,on="Date_hour_int",how="left")
    return df_meteo_merged

# define new objective fonction for xgboost
def fair_obj(preds, dtrain):
    """
    fair_obj function to optimize approximatively MAE
    """
    fair_constant = 30
    labels = dtrain.get_label()
    x = (preds - labels)
    den = abs(x) + fair_constant
    grad = fair_constant * x / (den)
    hess = fair_constant * fair_constant / (den * den)
    return grad, hess

def mean_absolute_err(y_true, y_pred):
    """
    MAE Metric
    """
    y_true = y_true
    y_pred= y_pred.get_label()
    return "MAE",np.mean(np.abs((y_true - y_pred))) 

def time_zone(df_meteo):
    """
    Aline the 2 time zone in the meteo file and production file
    """
    df_meteo["Date"] = df_meteo["Date"] + timedelta(hours=1)
    df_meteo_new = df_meteo[(df_meteo["fc_hor"]>=24)&(df_meteo["fc_hor"]<=47)]
    del(df_meteo)
    gc.collect()
    return df_meteo_new

def meteo_delta_feature(df_meteo):
    """
    generate meteo delta feature
    """
    df_meteo["vit_100_delta"] =df_meteo["vit_100"] - df_meteo["vit_100"].shift(1)
    df_meteo["vit_10_delta"] =df_meteo["vit_10"] - df_meteo["vit_10"].shift(1)
    df_meteo["dir_100_delta"] =df_meteo["dir_100"] - df_meteo["dir_100"].shift(1)
    df_meteo["dir_10_delta"] =df_meteo["dir_10"] - df_meteo["dir_10"].shift(1)
    return 

def submission_generation(res,save_adr):
    """
    generate submission file
    """
    submission = pd.read_csv("..//data//submit_benchmark.csv",sep=";")
    submission["Date"] = pd.to_datetime(submission["Date"],format = "%Y-%m-%d %H:%M:%S")
    add_feature_timestamp(submission)
    submission =  submission[["Date_hour_int","Eolienne","Date"]]
    submission=pd.merge(submission,res,on=["Date_hour_int","Eolienne"],how="left")[["Date","Eolienne","pred"]]
    submission.to_csv(save_adr,sep=';',header=True,index = False)

# DATA Loading

In [3]:
# parc data 
folder_adr = "..//data"
list_num_parc = [1,2,3]
list_date_parc = ["2015","2016","2017"]
list_col_parc_data_to_keep = ["Date","Eolienne","Production","Fonctionnement","Catégorie","State","Etat","Vent"]
parc_data_min = load_parc_data(folder_adr,list_num_parc,list_date_parc,list_col_parc_data_to_keep)

#forecast data
list_grille= [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
##for this example notebook, we will only take one "grille" (=9) 
df_meteo = create_df_meteo_from_list_grille(folder_adr,list_grille)

(12910771, 17)


# Feature engineering

In [4]:
# add feature and convert to hourly data
add_feature_timestamp(parc_data_min)
parc_data_hour =  min_to_hour(parc_data_min)
df_meteo = time_zone(df_meteo)
meteo_delta_feature(df_meteo)
add_feature_timestamp(df_meteo)

# we don't need the minute data any more
del(parc_data_min)
gc.collect()

eolienne_list = get_eolienne_list(parc_data_hour)
feature_list = ["vit_100","vit_10",'dir_100','dir_10',"vit_100_delta","vit_10_delta",'dir_100_delta','dir_10_delta',"fc_hor"]
join_key = ["Date_hour_int"]
meteo_grill_merged = meteo_grill_merge(df_meteo,feature_list,join_key)

In [5]:
parc_data_hour = pd.merge(parc_data_hour,meteo_grill_merged,on=["Date_hour_int"],how="left")

# Train/test validation

In [6]:
date_split_test = 2017010100
parc_data_hour_train = parc_data_hour[parc_data_hour["Date_hour_int"]<date_split_test]
parc_data_hour_test = parc_data_hour[parc_data_hour["Date_hour_int"]>=date_split_test]

parc_data_hour_train.dropna(inplace=True)
print(parc_data_hour_train.shape)
print(parc_data_hour_test.shape)

parc_data_hour_train.sort_values("Date_hour_int",inplace=True)
parc_data_hour_train_train = parc_data_hour_train.iloc[:94618,:]
parc_data_hour_train_test  = parc_data_hour_train.iloc[94618:189236,:]

(189214, 155)
(26884, 155)


C:\Users\IA2069\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\IA2069\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [7]:
# feature used in model 
col_target ="Production"
prefix_list = ["vit_100_","vit_10_","dir_100_","dir_10_","vit_100_delta_","vit_10_delta_","dir_100_delta_","dir_10_delta_"]
list_col_model =  []
list_grille= [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
for grille_ind in list_grille:
    list_col_model += [prefix+str(grille_ind) for prefix in prefix_list]

In [8]:
our_params = {'eta': 0.02,  'subsample': 0.99, 'colsample_bytree': 0.95, 
              'objective': 'reg:linear', 'max_depth':20, 'min_child_weight':20}

In [9]:
# validation
parc_data_hour_train_test["xgb_site"]=np.nan
for eol_name in eolienne_list:
    Eolienne_index = parc_data_hour_train_train["Eolienne"]==eol_name
    Eolienne_index_test = parc_data_hour_train_test["Eolienne"]==eol_name
    xgdtrain = xgb.DMatrix(parc_data_hour_train_train[Eolienne_index][list_col_model], parc_data_hour_train_train[Eolienne_index][col_target]) 
    xgdtest = xgb.DMatrix(parc_data_hour_train_test[Eolienne_index_test][list_col_model], parc_data_hour_train_test[Eolienne_index_test][col_target])
    evallist  = [(xgdtest,'eval'), (xgdtrain,'train')]
    print("mae optimiser ")
    xgb_model = xgb.train(params = our_params ,dtrain = xgdtrain, evals=evallist, num_boost_round = 250,verbose_eval=50,obj=fair_obj,feval=mean_absolute_err)
    print("rmse optimiser")
    xgb_model_2 = xgb.train(params = our_params ,dtrain = xgdtrain, evals=evallist, num_boost_round = 250,verbose_eval=50,feval=mean_absolute_err)
    parc_data_hour_train_test["xgb_site"][Eolienne_index_test]=xgb_model.predict(xgdtest)

C:\Users\IA2069\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


mae optimiser 
[0]	eval-rmse:660.277	train-rmse:738.831	eval-MAE:441.006	train-MAE:512.23
[50]	eval-rmse:311.194	train-rmse:338.933	eval-MAE:208.539	train-MAE:224.51
[100]	eval-rmse:281.09	train-rmse:295.728	eval-MAE:189.639	train-MAE:192.058
[150]	eval-rmse:276.468	train-rmse:283.676	eval-MAE:186.886	train-MAE:181.426
[200]	eval-rmse:275.332	train-rmse:277.664	eval-MAE:186.468	train-MAE:175.761
[249]	eval-rmse:274.77	train-rmse:273.168	eval-MAE:186.036	train-MAE:171.334
rmse optimiser
[0]	eval-rmse:683.578	train-rmse:764.11	eval-MAE:458.651	train-MAE:532.898
[50]	eval-rmse:353.901	train-rmse:348.395	eval-MAE:234.693	train-MAE:231.851
[100]	eval-rmse:285.481	train-rmse:203.022	eval-MAE:199.785	train-MAE:133.559
[150]	eval-rmse:277.866	train-rmse:146.871	eval-MAE:197.258	train-MAE:95.9882
[200]	eval-rmse:278.143	train-rmse:120.235	eval-MAE:198.071	train-MAE:78.0283
[249]	eval-rmse:279.042	train-rmse:106.544	eval-MAE:198.956	train-MAE:69.0293


C:\Users\IA2069\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\IA2069\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\IA2069\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(co

mae optimiser 
[0]	eval-rmse:653.023	train-rmse:712.699	eval-MAE:422.73	train-MAE:487.963
[50]	eval-rmse:364.607	train-rmse:380.793	eval-MAE:231.934	train-MAE:244.964
[100]	eval-rmse:337.107	train-rmse:340.408	eval-MAE:215.07	train-MAE:214.545
[150]	eval-rmse:333.148	train-rmse:330.23	eval-MAE:212.122	train-MAE:204.78
[200]	eval-rmse:333.124	train-rmse:326.376	eval-MAE:211.983	train-MAE:200.141
[249]	eval-rmse:333.131	train-rmse:323.222	eval-MAE:212.082	train-MAE:196.378
rmse optimiser
[0]	eval-rmse:666.762	train-rmse:726.486	eval-MAE:432.797	train-MAE:499.28
[50]	eval-rmse:392.98	train-rmse:351.751	eval-MAE:251.163	train-MAE:228.334
[100]	eval-rmse:343.656	train-rmse:219.784	eval-MAE:232.969	train-MAE:142.435
[150]	eval-rmse:338.956	train-rmse:165.415	eval-MAE:234.611	train-MAE:107.423
[200]	eval-rmse:340.148	train-rmse:139.613	eval-MAE:236.944	train-MAE:90.8755
[249]	eval-rmse:341.313	train-rmse:124.665	eval-MAE:238.353	train-MAE:81.3306
mae optimiser 
[0]	eval-rmse:673.162	train-rms

[50]	eval-rmse:319.292	train-rmse:303.197	eval-MAE:202.568	train-MAE:192.119
[100]	eval-rmse:264.312	train-rmse:176.744	eval-MAE:177.523	train-MAE:112.347
[150]	eval-rmse:258.789	train-rmse:128.525	eval-MAE:176.132	train-MAE:81.6472
[200]	eval-rmse:259.541	train-rmse:106.628	eval-MAE:177.317	train-MAE:67.9816
[249]	eval-rmse:260.48	train-rmse:95.0667	eval-MAE:178.335	train-MAE:60.8379
mae optimiser 
[0]	eval-rmse:644.802	train-rmse:725.562	eval-MAE:421.896	train-MAE:501.709
[50]	eval-rmse:352.025	train-rmse:365.608	eval-MAE:227.344	train-MAE:239.982
[100]	eval-rmse:332.56	train-rmse:327.189	eval-MAE:213.448	train-MAE:209.504
[150]	eval-rmse:331.522	train-rmse:318.12	eval-MAE:212.325	train-MAE:200.276
[200]	eval-rmse:331.953	train-rmse:314.185	eval-MAE:212.505	train-MAE:195.762
[249]	eval-rmse:332.156	train-rmse:310.813	eval-MAE:212.895	train-MAE:192.244
rmse optimiser
[0]	eval-rmse:658.987	train-rmse:740.734	eval-MAE:432.625	train-MAE:514.183
[50]	eval-rmse:379.672	train-rmse:353.375	e

# Prediction & make submission

In [11]:
res = parc_data_hour_test[["Date_hour_int","Eolienne"]]
res["pred"] =np.nan

xgdmat = xgb.DMatrix(parc_data_hour_train[list_col_model], parc_data_hour_train[col_target])
xgb_model = xgb.train(params = our_params ,dtrain = xgdmat, num_boost_round = 150,verbose_eval=10,obj=fair_obj,feval=mean_absolute_err)
res["pred"]=xgb_model.predict(xgb.DMatrix(parc_data_hour_test[list_col_model]))

save_adr = "..//submission//final_solution.csv"
submission_generation(res,save_adr)

C:\Users\IA2069\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\IA2069\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
